SQLite3のデータをダウンロード

In [1]:
!scp -i ~/.ssh/secret_keys/private_key.txt ubuntu@www.iiojun.com:/var/www/html/php/tlog_data.sqlite3 .

tlog_data.sqlite3                             100%   12KB 238.9KB/s   00:00    


タブ区切り形式で出力する

In [2]:
!sqlite3 tlog_data.sqlite3 < tlog.sql

Tweeファイルの読み込み

In [3]:
with open('BLA.twee', 'r') as f: lines = [ x. rstrip() for x in f.readlines() ]
lines.pop(0) # discard the first line
lines

['BLA',
 '',
 '',
 ':: StoryData',
 '{',
 '  "ifid": "84513303-33A0-470B-BC51-23DDDD00D2D1",',
 '  "format": "Snowman",',
 '  "format-version": "2.0.2",',
 '  "start": "Start",',
 '  "zoom": 0.6',
 '}',
 '',
 '',
 ':: Final page {"position":"600,1050","size":"100,100"}',
 'Your ID is <%= window.story.state.uid %>',
 '',
 'Logged data is as follows:',
 '',
 '<%= JSON.stringify(window.story.state.hash) %>',
 '',
 '<%',
 '  $.ajax({',
 "    url: 'https://www.iiojun.com/php/tlog.php',",
 "    type: 'POST',",
 "    data: { 'uid': window.story.state.uid,",
 "                'log': JSON.stringify(window.story.state.hash) },",
 "    dataType: 'json'",
 '  });',
 '%>',
 '',
 '',
 ':: Goal {"position":"600,900","size":"100,100"}',
 'We are in the Goal.',
 '',
 'Go to [[Final page]]',
 '',
 '',
 ':: Page A {"position":"400,450","size":"100,100"}',
 'We\'re in "Page A"',
 '',
 'Go to the [[The 2nd Question]]',
 '',
 '',
 ':: Page B {"position":"600,450","size":"100,100"}',
 'We\'re in "Page B"',
 

:: StoryTitle　{...} から StoryTitleを，　[[Link]]　からLinkを抜き出す

In [4]:
import re

parent_children = {}

for line in lines:
    if re.match('^:: .* \{.*\}', line): # タイトル行
        title = line.split('{')[0][3:].rstrip() # 冒頭の「:: 」と後ろのスペースを削除
        parent_children[title] = []
    if re.match('.*\[\[.*\]\].*', line): # リンクを含む行
        for child in re.findall(r'\[\[(.*?)\]\]', line): parent_children[title].append(child)

parent_children

{'Final page': [],
 'Goal': ['Final page'],
 'Page A': ['The 2nd Question'],
 'Page B': ['The 2nd Question'],
 'Page C': ['The 2nd Question'],
 'Page D': ['Goal'],
 'Page E': ['Goal'],
 'Page F': ['Goal'],
 'Start': ['Page A', 'Page B', 'Page C'],
 'The 2nd Question': ['Page D', 'Page E', 'Page F']}

TSVで表現されたログデータを読み込み，データフレームを作成する

In [5]:
import pandas as pd
with open('tlog.tsv', 'r') as f: lines = [ line.split('\t') for line in f.readlines() ]
df = pd.DataFrame(lines, columns=['id', 'uid', 'log'])[['uid','log']]
df

,uid,log
0,18cb876005e55,"{""Start"":1703901790308,""Page A"":1703901795881,..."
1,18cb876510c36f,"{""Start"":1703901810956,""Page B"":1703901812357,..."
2,18cb876780a254,"{""Start"":1703901820939,""Page C"":1703901822614,..."
3,18cb9cb720a3b9,"{""Start"":1703924167185,""Page B"":1703924169094,..."
4,18cb9cbb0b02b6,"{""Start"":1703924183217,""Page C"":1703924184509,..."
5,18cb9cbd0a22c7,"{""Start"":1703924191395,""Page C"":1703924192303,..."
6,18cb9cbf9bb321,"{""Start"":1703924201921,""Page A"":1703924203155,..."
7,18cb9cc13e0288,"{""Start"":1703924208609,""Page A"":1703924210372,..."
8,18cb9cc3e50399,"{""Start"":1703924219473,""Page A"":1703924220953,..."
9,18cb9d6b9aa236,"{""Start"":1703924906416,""Page C"":1703924907652,..."


logカラムをJSON文字列からオブジェクトに変換する

In [6]:
import json
df['log'] = df['log'].apply(lambda x: json.loads(x))
df

,uid,log
0,18cb876005e55,"{'Start': 1703901790308, 'Page A': 17039017958..."
1,18cb876510c36f,"{'Start': 1703901810956, 'Page B': 17039018123..."
2,18cb876780a254,"{'Start': 1703901820939, 'Page C': 17039018226..."
3,18cb9cb720a3b9,"{'Start': 1703924167185, 'Page B': 17039241690..."
4,18cb9cbb0b02b6,"{'Start': 1703924183217, 'Page C': 17039241845..."
5,18cb9cbd0a22c7,"{'Start': 1703924191395, 'Page C': 17039241923..."
6,18cb9cbf9bb321,"{'Start': 1703924201921, 'Page A': 17039242031..."
7,18cb9cc13e0288,"{'Start': 1703924208609, 'Page A': 17039242103..."
8,18cb9cc3e50399,"{'Start': 1703924219473, 'Page A': 17039242209..."
9,18cb9d6b9aa236,"{'Start': 1703924906416, 'Page C': 17039249076..."


{'A' : 123, 'B' : 789, 'C' : 456 } というようなDictをvalueでソートして[('A', 123), ('C', 456), ('B', 789)]というようなリストに変換する例

In [7]:
x = {'A' : 123, 'B' : 789, 'C' : 456 }
sorted(x.items(), key=lambda i: i[1])

[('A', 123), ('C', 456), ('B', 789)]

これを利用して，logカラムをタイムスタンプ順に並んだリストにする

In [8]:
df['log'] = df['log'].apply(lambda x: sorted(x.items(), key=lambda i: i[1]))
df

,uid,log
0,18cb876005e55,"[(Start, 1703901790308), (Page A, 170390179588..."
1,18cb876510c36f,"[(Start, 1703901810956), (Page B, 170390181235..."
2,18cb876780a254,"[(Start, 1703901820939), (Page C, 170390182261..."
3,18cb9cb720a3b9,"[(Start, 1703924167185), (Page B, 170392416909..."
4,18cb9cbb0b02b6,"[(Start, 1703924183217), (Page C, 170392418450..."
5,18cb9cbd0a22c7,"[(Start, 1703924191395), (Page C, 170392419230..."
6,18cb9cbf9bb321,"[(Start, 1703924201921), (Page A, 170392420315..."
7,18cb9cc13e0288,"[(Start, 1703924208609), (Page A, 170392421037..."
8,18cb9cc3e50399,"[(Start, 1703924219473), (Page A, 170392422095..."
9,18cb9d6b9aa236,"[(Start, 1703924906416), (Page C, 170392490765..."


[('A', 123), ('C', 666), ('B', 789)]のそれぞれの項の2つめの値の差分を計算し，{'A': 543, 'C': 123,'B': -1}となるように変換する（最後は差分を計算できないので，-1とする）

In [9]:
x = {'A' : 123, 'B' : 789, 'C' : 666 }
x = sorted(x.items(), key=lambda i: i[1])
z = [list(y) for y in x]
for i in range(len(z)-1):
    z[i][1] = z[i+1][1]-z[i][1]
z[-1][1] = -1
dict(z)

{'A': 543, 'C': 123, 'B': -1}

関数化する

In [10]:
def convert(x):
    x = [list(y) for y in x]
    for i in range(len(x)-1): 
        x[i][1] = x[i+1][1]-x[i][1]
    x[-1][1] = -1
    return dict(x)

df['log']に適用する

In [11]:
df['log'] = df['log'].apply(convert)
df

,uid,log
0,18cb876005e55,"{'Start': 5573, 'Page A': 1318, 'The 2nd Quest..."
1,18cb876510c36f,"{'Start': 1401, 'Page B': 1021, 'The 2nd Quest..."
2,18cb876780a254,"{'Start': 1675, 'Page C': 963, 'The 2nd Questi..."
3,18cb9cb720a3b9,"{'Start': 1909, 'Page B': 1150, 'The 2nd Quest..."
4,18cb9cbb0b02b6,"{'Start': 1292, 'Page C': 933, 'The 2nd Questi..."
5,18cb9cbd0a22c7,"{'Start': 908, 'Page C': 1908, 'The 2nd Questi..."
6,18cb9cbf9bb321,"{'Start': 1234, 'Page A': 906, 'The 2nd Questi..."
7,18cb9cc13e0288,"{'Start': 1763, 'Page A': 3980, 'The 2nd Quest..."
8,18cb9cc3e50399,"{'Start': 1480, 'Page A': 2435, 'The 2nd Quest..."
9,18cb9d6b9aa236,"{'Start': 1236, 'Page C': 781, 'The 2nd Questi..."


各ノードの滞在時間平均を求めるためのログを出力する

In [12]:
logs = list(df['log'])
logs

[{'Start': 5573,
  'Page A': 1318,
  'The 2nd Question': 1652,
  'Page E': 1202,
  'Goal': -1},
 {'Start': 1401,
  'Page B': 1021,
  'The 2nd Question': 875,
  'Page F': 1254,
  'Goal': -1},
 {'Start': 1675,
  'Page C': 963,
  'The 2nd Question': 1271,
  'Page D': 839,
  'Goal': -1},
 {'Start': 1909,
  'Page B': 1150,
  'The 2nd Question': 1294,
  'Page F': 1264,
  'Goal': -1},
 {'Start': 1292,
  'Page C': 933,
  'The 2nd Question': 719,
  'Page F': 1029,
  'Goal': -1},
 {'Start': 908,
  'Page C': 1908,
  'The 2nd Question': 1896,
  'Page D': 2598,
  'Goal': -1},
 {'Start': 1234,
  'Page A': 906,
  'The 2nd Question': 694,
  'Page D': 868,
  'Goal': -1},
 {'Start': 1763,
  'Page A': 3980,
  'The 2nd Question': 1332,
  'Page F': 1004,
  'Goal': -1},
 {'Start': 1480,
  'Page A': 2435,
  'The 2nd Question': 2225,
  'Page D': 736,
  'Goal': -1},
 {'Start': 1236,
  'Page C': 781,
  'The 2nd Question': 534,
  'Page F': 1126,
  'Goal': -1},
 {'Start': 1172,
  'Page C': 744,
  'The 2nd Questio

各ノードの滞在時間の平均値を求める

In [13]:
import numpy as np

class Node:
    def __init__(self, label, children):
        self.label = label
        self.seconds = []
        self.to_nodes = { x: 0 for x in children }
        
    def add(self, second):
        self.seconds.append(second)

    def mean(self): # 平均値を求める．ただし，異常データは除く
        secs = []
        for sec in self.seconds:
            if sec < 300000: secs.append(sec) # 5分以上かかったものは削除     
        return 0 if len(secs) == 0 else np.mean(secs)

    def add_to_node(self, to_node):
        label = to_node.label
        self.to_nodes[label] += 1

            
nodes = {}

# twee ファイルからノードのリストを作成する

for label in parent_children.keys(): nodes[label] = Node(label, parent_children[label])

for log in logs:
    for label in log.keys():
        nodes[label].add(log[label])

# 各ノードの滞在時間の平均値を求める
means = { k : nodes[k].mean() for k in nodes }

# 最大値で正規化する
max_duration = max(means.values())
means_norm = { k : means[k] / max_duration for k in means }

means_norm['Goal'] = 0.0

means_norm

{'Final page': 0.0,
 'Goal': 0.0,
 'Page A': 0.5219308844852586,
 'Page B': 1.0,
 'Page C': 0.24295150636378285,
 'Page D': 0.30455534074432095,
 'Page E': 0.9734171097148381,
 'Page F': 0.2546778982255058,
 'Start': 0.398760615894497,
 'The 2nd Question': 0.27071394048194436}

遷移状況を求める

In [14]:
for log in logs:
    for f_node in log.keys():
        from_node = nodes[f_node]
        for to_node in from_node.to_nodes:
            if to_node in log.keys(): from_node.add_to_node(nodes[to_node])

ドットスクリプトを出力

In [15]:
import matplotlib.cm as cm

def get_color(ratio): # ratio は 0.0 ~ 1.0 までの値をとる
    if ratio == 0.0: return "ffffff"
    r,g,b,a = cm.rainbow(ratio)
    return format(int(r*255), '02x')+format(int(g*255), '02x')+format(int(b*255), '02x')

with open('graph.dot', 'w') as f:
    f.write('digraph graph_name {\n')
    f.write('''
graph [
    charset = "UTF-8";
    labelloc = "t",
    labeljust = "c",
    bgcolor = white,
    fontcolor = black,
    fontsize = 18,
    style = "filled",
    rankdir = TB,
    margin = 0.5,
    layout = dot
  ];
''')
    f.write('// nodes\n')
    for node in nodes:
        if node == 'Final page': continue # 最終のダミーページは対象外
        f.write(f'  "{node} ({len(nodes[node].seconds)})" [ style = \"solid, filled\"; fillcolor = \"#{get_color(means_norm[node])}\"];\n')
    f.write('// arcs\n')
    for node in nodes:
        to_node_hash = nodes[node].to_nodes
        for to_node in to_node_hash:
            if to_node == 'Final page': continue # 最終のダミーページは対象外
            f.write(f'  "{node} ({len(nodes[node].seconds)})" -> "{to_node} ({len(nodes[to_node].seconds)})" [label = {to_node_hash[to_node]}];\n')
    f.write('}\n')

出力したドットスクリプトからPNGイメージを作成

In [16]:
!dot graph.dot -o graph.png -Tpng
!open graph.png